In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, confusion_matrix, classification_report
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
# Read and preprocess the data
train_data = pd.read_csv('train (1).csv')
train_data = train_data.fillna(0)
cat_cols = ['Artist Name', 'Track Name']
for col in cat_cols:
    train_data[col] = train_data[col].astype('str')

# Handle missing values


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(train_data.drop('Class', axis=1), train_data['Class'], test_size=0.2, random_state=42)
# Data preprocessing


# Train the CatBoost model
model = CatBoostClassifier(loss_function='MultiClass', verbose=False)
model.fit(X_train, y_train, cat_features=cat_cols)

# Evaluate the model
y_pred_train = model.predict_proba(X_train)
y_pred_test = model.predict_proba(X_test)
train_log_loss = log_loss(y_train, y_pred_train)
test_log_loss = log_loss(y_test, y_pred_test)
print(f"Train log loss: {train_log_loss}")
print(f"Test log loss: {test_log_loss}")

# Generate predictions for the test data
test_data = pd.read_csv('test (2).csv')
test_data = test_data.fillna(0)
for col in cat_cols:
    test_data[col] = test_data[col].astype('str')
# Handle missing values


y_pred = model.predict_proba(test_data)
max_values = y_pred.argmax(axis=1)
test_data['Class'] = max_values
test_data.to_csv('adjusted_predictions.csv', index=False)

# Additional evaluation
y_pred_train_labels = model.predict(X_train)
y_pred_test_labels = model.predict(X_test)
train_classification_report = classification_report(y_train, y_pred_train_labels)
test_classification_report = classification_report(y_test, y_pred_test_labels)
print("Train Classification Report:")
print(train_classification_report)
print("Test Classification Report:")
print(test_classification_report)
# Create a submission file DataFrame based on the sample submission format
submission_file = pd.read_csv('sample_submission (2).csv')
submission_file['Class'] = max_values

# Save the submission file
submission_file.to_csv('submission88.csv', index=False)

#score 0.678

In [ ]:




import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, confusion_matrix, classification_report
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
# Read and preprocess the data
raw_data = pd.read_csv('train (1).csv')

cat_cols = ['Artist Name', 'Track Name']
for col in cat_cols:
    raw_data[col] = raw_data[col].astype('str')

# Handle missing values
raw_data["Popularity"].fillna(raw_data["Popularity"].mean(), inplace=True)
raw_data["key"].fillna(raw_data["key"].mode()[0], inplace=True)
raw_data["instrumentalness"].fillna(0, inplace=True)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(raw_data.drop('Class', axis=1), raw_data['Class'], test_size=0.2, random_state=42)
# Data preprocessing


# Train the CatBoost model
model = CatBoostClassifier(loss_function='MultiClass', verbose=False)
model.fit(X_train, y_train, cat_features=cat_cols)

# Evaluate the model
y_pred_train = model.predict_proba(X_train)
y_pred_test = model.predict_proba(X_test)
train_log_loss = log_loss(y_train, y_pred_train)
test_log_loss = log_loss(y_test, y_pred_test)
print(f"Train log loss: {train_log_loss}")
print(f"Test log loss: {test_log_loss}")

# Generate predictions for the test data
test_data = pd.read_csv('test (2).csv')
test_data = test_data.fillna(0)
for col in cat_cols:
    test_data[col] = test_data[col].astype('str')
# Handle missing values
test_data["Popularity"].fillna(test_data["Popularity"].mean(), inplace=True)
test_data["key"].fillna(test_data["key"].mode()[0], inplace=True)
test_data["instrumentalness"].fillna(0, inplace=True)

y_pred = model.predict_proba(test_data)
max_values = y_pred.argmax(axis=1)
test_data['Class'] = max_values
test_data.to_csv('adjusted_predictions.csv', index=False)

# Additional evaluation
y_pred_train_labels = model.predict(X_train)
y_pred_test_labels = model.predict(X_test)
train_classification_report = classification_report(y_train, y_pred_train_labels)
test_classification_report = classification_report(y_test, y_pred_test_labels)
print("Train Classification Report:")
print(train_classification_report)
print("Test Classification Report:")
print(test_classification_report)


# Make predictions on the test data
test_preds =  model.predict(test_data)

# Create a submission file DataFrame based on the sample submission format
submission_file = pd.read_csv('sample_submission (2).csv')
submission_file['Class'] = test_preds

# Save the submission file
submission_file.to_csv('submission681.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, accuracy_score, classification_report
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Load the training data
train_set = pd.read_csv("train (1).csv")
test_data = pd.read_csv('test (2).csv')

# Drop irrelevant columns
train_set.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)
test_data.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)

# Separate features and target variable
X = train_set.drop(columns=["Class"])
y = train_set["Class"]

# Data preprocessing
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)
test_data_imputed = imputer.transform(test_data)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
test_data_scaled = scaler.transform(test_data_imputed)

# Feature engineering
X_scaled = np.concatenate([X_scaled, np.square(X_scaled)], axis=1)
test_data_scaled = np.concatenate([test_data_scaled, np.square(test_data_scaled)], axis=1)

# Create new features based on the the index of columns
X_scaled = np.concatenate([X_scaled, X_scaled[:, 3:4] * X_scaled[:, 5:6]], axis=1)
test_data_scaled = np.concatenate([test_data_scaled, test_data_scaled[:, 3:4] * test_data_scaled[:, 5:6]], axis=1)
X_scaled = np.concatenate([X_scaled, X_scaled[:, 2:3] * X_scaled[:, 6:7]], axis=1)
test_data_scaled = np.concatenate([test_data_scaled, test_data_scaled[:, 2:3] * test_data_scaled[:, 6:7]], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.30, random_state=42)

# Initialize classifiers with hyperparameter tuning
rf_clf = RandomForestClassifier(n_estimators=200, max_depth=30, random_state=42, class_weight='balanced')
lgbm_clf = LGBMClassifier(n_estimators=200, max_depth=30, random_state=42)
cat_clf = CatBoostClassifier(random_state=42, verbose=0)
extra_trees_clf = ExtraTreesClassifier(n_estimators=200, max_depth=30, random_state=42)

# Stacking Classifier with RandomForest, LGBM, CatBoost, and ExtraTrees as base models
# and RandomForest as the meta-classifier
stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_clf),
        ('lgbm', lgbm_clf),
        ('cat', cat_clf),
        ('extra_trees', extra_trees_clf)
    ],
    final_estimator=RandomForestClassifier(random_state=42),
    stack_method='predict_proba'  # Use predict_proba for meta-features
)

# Train the stacking classifier
stacking_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = stacking_clf.predict(X_test)

# Calculate F1 score
f1_macro = f1_score(y_test, y_pred, average='macro')
print("F1 score (macro):", f1_macro)

# Generate classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

# Scale the test data using the trained scaler
test_data_scaled = scaler.transform(test_data_imputed)

# Feature engineering
test_data_scaled = np.concatenate([test_data_scaled, np.square(test_data_scaled)], axis=1)
test_data_scaled = np.concatenate([test_data_scaled, test_data_scaled[:, 3:4] * test_data_scaled[:, 5:6]], axis=1)
test_data_scaled = np.concatenate([test_data_scaled, test_data_scaled[:, 2:3] * test_data_scaled[:, 6:7]], axis=1)

# Make predictions on the test data
test_preds = stacking_clf.predict(test_data_scaled)

# Create a submission file DataFrame based on the sample submission format
submission_file = pd.read_csv('sample_submission (2).csv')
submission_file['Class'] = test_preds

# Save the submission file
submission_file.to_csv('submission571.csv', index=False)
#score 58

F1 score (macro): 0.6250223855669649
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.81      0.77       160
           1       0.47      0.26      0.33       315
           2       0.55      0.40      0.46       327
           3       0.87      0.76      0.81       100
           4       0.67      0.63      0.65       105
           5       0.73      0.73      0.73       361
           6       0.46      0.42      0.44       610
           7       0.94      0.94      0.94       125
           8       0.65      0.59      0.62       435
           9       0.53      0.53      0.53       595
          10       0.53      0.67      0.59      1186

    accuracy                           0.58      4319
   macro avg       0.65      0.61      0.63      4319
weighted avg       0.58      0.58      0.57      4319



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# Load the training data
train_set = pd.read_csv("train (1).csv")
test_data = pd.read_csv('test (2).csv')

# Drop irrelevant columns
train_set.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)
test_data.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)

# Separate features and target variable
X = train_set.drop(columns=["Class"])
y = train_set["Class"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Feature engineering
# You can create new features based on the given information
X_train['loudness_energy_ratio'] = X_train['loudness'] / X_train['energy']
X_test['loudness_energy_ratio'] = X_test['loudness'] / X_test['energy']

# Data preprocessing
imputer = SimpleImputer(strategy="mean")
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Initialize classifiers with hyperparameter tuning
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
param_grid_rf = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_rf = GridSearchCV(rf_clf, param_grid_rf, cv=3, scoring='f1_macro')
grid_rf.fit(X_train_scaled, y_train)
rf_clf = grid_rf.best_estimator_

lgbm_clf = LGBMClassifier(random_state=42)
param_grid_lgbm = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_lgbm = GridSearchCV(lgbm_clf, param_grid_lgbm, cv=3, scoring='f1_macro')
grid_lgbm.fit(X_train_scaled, y_train)
lgbm_clf = grid_lgbm.best_estimator_

cat_clf = CatBoostClassifier(random_state=42, verbose=0)

extra_trees_clf = ExtraTreesClassifier(random_state=42)
param_grid_extra_trees = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}

grid_extra_trees = GridSearchCV(extra_trees_clf, param_grid_extra_trees, cv=3, scoring='f1_macro')
grid_extra_trees.fit(X_train_scaled, y_train)
extra_trees_clf = grid_extra_trees.best_estimator_

# Stacking Classifier with RandomForest, LGBM, CatBoost, and ExtraTrees as base models
# and RandomForest as the meta-classifier
stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_clf),
        ('lgbm', lgbm_clf),
        ('cat', cat_clf),
        ('extra_trees', extra_trees_clf)
    ],
    final_estimator=RandomForestClassifier(random_state=42),
    stack_method='predict_proba'  # Use predict_proba for meta-features
)

# Train the stacking classifier
stacking_clf.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = stacking_clf.predict(X_test_scaled)

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='macro')
print("F1 score:", f1)
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

F1 score: 0.6231137565277939
Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.79      0.75       160
           1       0.44      0.23      0.30       315
           2       0.56      0.46      0.51       327
           3       0.86      0.74      0.80       100
           4       0.68      0.64      0.66       105
           5       0.74      0.72      0.73       361
           6       0.45      0.41      0.43       610
           7       0.94      0.94      0.94       125
           8       0.63      0.58      0.60       435
           9       0.56      0.56      0.56       595
          10       0.52      0.66      0.58      1186

    accuracy                           0.58      4319
   macro avg       0.65      0.61      0.62      4319
weighted avg       0.57      0.58      0.57      4319



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# Load the training data
train_set = pd.read_csv("train (1).csv")
test_data = pd.read_csv('test (2).csv')

# Drop irrelevant columns
train_set.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)
test_data.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)

# Separate features and target variable
X = train_set.drop(columns=["Class"])
y = train_set["Class"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Data preprocessing
imputer = SimpleImputer(strategy="mean")
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Feature engineering
X_train_fe = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_fe = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Add new features based on the given information
X_train_fe["popularity_energy_ratio"] = X_train_fe["Popularity"] / X_train_fe["energy"]
X_train_fe["danceability_tempo_product"] = X_train_fe["danceability"] * X_train_fe["tempo"]

X_test_fe["popularity_energy_ratio"] = X_test_fe["Popularity"] / X_test_fe["energy"]
X_test_fe["danceability_tempo_product"] = X_test_fe["danceability"] * X_test_fe["tempo"]

# Initialize classifiers with hyperparameter tuning
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
param_grid_rf = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_rf = GridSearchCV(rf_clf, param_grid_rf, cv=3, scoring='f1_macro')
grid_rf.fit(X_train_fe, y_train)
rf_clf = grid_rf.best_estimator_

lgbm_clf = LGBMClassifier(random_state=42)
param_grid_lgbm = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_lgbm = GridSearchCV(lgbm_clf, param_grid_lgbm, cv=3, scoring='f1_macro')
grid_lgbm.fit(X_train_fe, y_train)
lgbm_clf = grid_lgbm.best_estimator_

cat_clf = CatBoostClassifier(random_state=42, verbose=0)

extra_trees_clf = ExtraTreesClassifier(random_state=42)
param_grid_extra_trees = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_extra_trees = GridSearchCV(extra_trees_clf, param_grid_extra_trees, cv=3, scoring='f1_macro')
grid_extra_trees.fit(X_train_fe, y_train)
extra_trees_clf = grid_extra_trees.best_estimator_

# Train the stacking classifier
stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_clf),
        ('lgbm', lgbm_clf),
        ('cat', cat_clf),
        ('extra_trees', extra_trees_clf)
    ],
    final_estimator=RandomForestClassifier(random_state=42),
    stack_method='predict_proba'  # Use predict_proba for meta-features
)

# Train the stacking classifier
stacking_clf.fit(X_train_fe, y_train)

# Make predictions on the test set
y_pred = stacking_clf.predict(X_test_fe)

class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)

Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.79      0.76       160
           1       0.49      0.24      0.32       315
           2       0.57      0.42      0.48       327
           3       0.85      0.71      0.77       100
           4       0.66      0.63      0.64       105
           5       0.73      0.75      0.74       361
           6       0.46      0.42      0.44       610
           7       0.92      0.94      0.93       125
           8       0.62      0.58      0.60       435
           9       0.55      0.55      0.55       595
          10       0.52      0.66      0.58      1186

    accuracy                           0.57      4319
   macro avg       0.64      0.61      0.62      4319
weighted avg       0.57      0.57      0.57      4319



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

train=pd.read_csv('train (1).csv')
test=pd.read_csv('test (2).csv')
submission_file=('sample_submission (2).csv')
train.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)
test.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)




train["Popularity"].fillna(train["Popularity"].mean(), inplace=True)
train["key"].fillna(train["key"].mode()[0], inplace=True)
train["instrumentalness"].fillna(0, inplace=True)


# Create a copy of the train DataFrame
test["Popularity"].fillna(test["Popularity"].mean(), inplace=True)
test["key"].fillna(test["key"].mode()[0], inplace=True)
test["instrumentalness"].fillna(0, inplace=True)

# Separate features and target variable
X = train.drop(columns=["Class"])
y = train["Class"]
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Initialize classifiers with hyperparameter tuning
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
param_grid_rf = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_rf = GridSearchCV(rf_clf, param_grid_rf, cv=3, scoring='accuracy')
grid_rf.fit(X_train_scaled, y_train)
rf_clf = grid_rf.best_estimator_

lgbm_clf = LGBMClassifier(random_state=42)
param_grid_lgbm = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30]
}
grid_lgbm = GridSearchCV(lgbm_clf, param_grid_lgbm, cv=3, scoring='accuracy')
grid_lgbm.fit(X_train_scaled, y_train)
lgbm_clf = grid_lgbm.best_estimator_

cat_clf = CatBoostClassifier(random_state=42, verbose=0)

extra_trees_clf = ExtraTreesClassifier(random_state=42)
param_grid_extra_trees = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_extra_trees = GridSearchCV(extra_trees_clf, param_grid_extra_trees, cv=3, scoring='accuracy')
grid_extra_trees.fit(X_train_scaled, y_train)
extra_trees_clf = grid_extra_trees.best_estimator_

# Stacking Classifier with RandomForest, LGBM, CatBoost, and ExtraTrees as base models
# and RandomForest as the meta-classifier
stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_clf),
        ('lgbm', lgbm_clf),
        ('cat', cat_clf),
        ('extra_trees', extra_trees_clf)
    ],
    final_estimator=RandomForestClassifier(random_state=42),
    stack_method='predict_proba'  # Use predict_proba for meta-features
)
# Train the stacking classifier
stacking_clf.fit(X_train_scaled, y_train)
# Make predictions on the test set
y_pred = stacking_clf.predict(X_test_scaled)


# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy score:", accuracy)

# Impute missing values in the test data using the trained imputer


# Scale the test data using the trained scaler
test_data_scaled = scaler.transform(test)

# Make predictions on the test data
test_preds = stacking_clf.predict(test_data_scaled)

# Create a submission file DataFrame based on the sample submission format
submission_file = pd.read_csv('sample_submission (2).csv')
submission_file['Class'] = test_preds

# Save the submission file
submission_file.to_csv('submission571.csv', index=False)



Best Model Accuracy on Validation Set: 0.5451388888888888


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

# Load the training data
train_set = pd.read_csv("train (1).csv")
test_data = pd.read_csv('test (2).csv')

# Drop irrelevant columns
train_set.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)
test_data.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)

# Separate features and target variable
X = train_set.drop(columns=["Class"])
y = train_set["Class"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Data preprocessing
imputer = SimpleImputer(strategy="mean")
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Feature engineering
X_train_fe = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_fe = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Add new features based on the given information
X_train_fe["popularity_energy_ratio"] = X_train_fe["Popularity"] / X_train_fe["energy"]
X_train_fe["danceability_tempo_product"] = X_train_fe["danceability"] * X_train_fe["tempo"]

X_test_fe["popularity_energy_ratio"] = X_test_fe["Popularity"] / X_test_fe["energy"]
X_test_fe["danceability_tempo_product"] = X_test_fe["danceability"] * X_test_fe["tempo"]

# Initialize classifiers with hyperparameter tuning
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
param_grid_rf = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_rf = GridSearchCV(rf_clf, param_grid_rf, cv=3, scoring='f1_macro')
grid_rf.fit(X_train_fe, y_train)
rf_clf = grid_rf.best_estimator_

lgbm_clf = LGBMClassifier(random_state=42)
param_grid_lgbm = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_lgbm = GridSearchCV(lgbm_clf, param_grid_lgbm, cv=3, scoring='f1_macro')
grid_lgbm.fit(X_train_fe, y_train)
lgbm_clf = grid_lgbm.best_estimator_

cat_clf = CatBoostClassifier(random_state=42, verbose=0)

extra_trees_clf = ExtraTreesClassifier(random_state=42)
param_grid_extra_trees = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_extra_trees = GridSearchCV(extra_trees_clf, param_grid_extra_trees, cv=3, scoring='f1_macro')
grid_extra_trees.fit(X_train_fe, y_train)
extra_trees_clf = grid_extra_trees.best_estimator_

# Train the stacking classifier
stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_clf),
        ('lgbm', lgbm_clf),
        ('cat', cat_clf),
        ('extra_trees', extra_trees_clf)
    ],
    final_estimator=RandomForestClassifier(random_state=42),
    stack_method='predict_proba'  # Use predict_proba for meta-features
)

# Train the stacking classifier
stacking_clf.fit(X_train_fe, y_train)

# Make predictions on the test set
y_pred = stacking_clf.predict(X_test_fe)

class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)


Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.79      0.76       160
           1       0.49      0.24      0.32       315
           2       0.57      0.42      0.48       327
           3       0.85      0.71      0.77       100
           4       0.66      0.63      0.64       105
           5       0.73      0.75      0.74       361
           6       0.46      0.42      0.44       610
           7       0.92      0.94      0.93       125
           8       0.62      0.58      0.60       435
           9       0.55      0.55      0.55       595
          10       0.52      0.66      0.58      1186

    accuracy                           0.57      4319
   macro avg       0.64      0.61      0.62      4319
weighted avg       0.57      0.57      0.57      4319



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

train = pd.read_csv('train (1).csv')
test = pd.read_csv('test (2).csv')

train.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)
test.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)

train["Popularity"].fillna(train["Popularity"].mean(), inplace=True)
train["key"].fillna(train["key"].mode()[0], inplace=True)
train["instrumentalness"].fillna(0, inplace=True)

test["Popularity"].fillna(test["Popularity"].mean(), inplace=True)
test["key"].fillna(test["key"].mode()[0], inplace=True)
test["instrumentalness"].fillna(0, inplace=True)

# Separate features and target variable
X = train.drop(columns=["Class"])
y = train["Class"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize classifiers with hyperparameter tuning
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
lgbm_clf = LGBMClassifier(random_state=42)
cat_clf = CatBoostClassifier(loss_function='MultiClass', verbose=False)
extra_trees_clf = ExtraTreesClassifier(random_state=42)

# Train the classifiers
rf_clf.fit(X_train_scaled, y_train)
lgbm_clf.fit(X_train_scaled, y_train)
cat_clf.fit(X_train_scaled, y_train)
extra_trees_clf.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred_rf = rf_clf.predict(X_test_scaled)
y_pred_lgbm = lgbm_clf.predict(X_test_scaled)
y_pred_cat = cat_clf.predict(X_test_scaled)
y_pred_extra_trees = extra_trees_clf.predict(X_test_scaled)

# Classification report for each model
print("Random Forest Classifier:")
print(classification_report(y_test, y_pred_rf))
print("-------------------------------------------------------")
print("LGBM Classifier:")
print(classification_report(y_test, y_pred_lgbm))
print("-------------------------------------------------------")
print("CatBoost Classifier:")
print(classification_report(y_test, y_pred_cat))
print("-------------------------------------------------------")
print("Extra Trees Classifier:")
print(classification_report(y_test, y_pred_extra_trees))
print("-------------------------------------------------------")


Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.69      0.78      0.73       175
           1       0.03      0.01      0.02       339
           2       0.59      0.36      0.44       363
           3       0.81      0.67      0.73       108
           4       0.70      0.60      0.65       118
           5       0.72      0.69      0.71       397
           6       0.36      0.29      0.32       689
           7       0.91      0.93      0.92       137
           8       0.62      0.52      0.57       484
           9       0.48      0.52      0.50       646
          10       0.44      0.62      0.52      1295

    accuracy                           0.51      4751
   macro avg       0.58      0.55      0.56      4751
weighted avg       0.50      0.51      0.49      4751

-------------------------------------------------------
LGBM Classifier:
              precision    recall  f1-score   support

           0       0.72      0.75

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

# Load the training data
train_set = pd.read_csv("train (1).csv")
test_data = pd.read_csv('test (2).csv')

# Drop irrelevant columns
train_set.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)
test_data.drop(columns=["Id", "Track Name", "Artist Name"], inplace=True)

# Separate features and target variable
X = train_set.drop(columns=["Class"])
y = train_set["Class"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Data preprocessing
imputer = SimpleImputer(strategy="mean")
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Feature engineering
X_train_fe = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_fe = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Add new features based on the given information
X_train_fe["popularity_energy_ratio"] = X_train_fe["Popularity"] / X_train_fe["energy"]
X_train_fe["danceability_tempo_product"] = X_train_fe["danceability"] * X_train_fe["tempo"]

X_test_fe["popularity_energy_ratio"] = X_test_fe["Popularity"] / X_test_fe["energy"]
X_test_fe["danceability_tempo_product"] = X_test_fe["danceability"] * X_test_fe["tempo"]

# Initialize classifiers with hyperparameter tuning
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
param_grid_rf = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_rf = GridSearchCV(rf_clf, param_grid_rf, cv=3, scoring='f1_macro')
grid_rf.fit(X_train_fe, y_train)
rf_clf = grid_rf.best_estimator_

lgbm_clf = LGBMClassifier(random_state=42)
param_grid_lgbm = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_lgbm = GridSearchCV(lgbm_clf, param_grid_lgbm, cv=3, scoring='f1_macro')
grid_lgbm.fit(X_train_fe, y_train)
lgbm_clf = grid_lgbm.best_estimator_

cat_clf = CatBoostClassifier(random_state=42, verbose=0)
cat_clf.fit(X_train_fe, y_train)  # Fit the CatBoost classifier

extra_trees_clf = ExtraTreesClassifier(random_state=42)
param_grid_extra_trees = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20, 30]
}
grid_extra_trees = GridSearchCV(extra_trees_clf, param_grid_extra_trees, cv=3, scoring='f1_macro')
grid_extra_trees.fit(X_train_fe, y_train)
extra_trees_clf = grid_extra_trees.best_estimator_

# Train the stacking classifier
stacking_clf = StackingClassifier(
    estimators=[
        ('rf', rf_clf),
        ('lgbm', lgbm_clf),
        ('cat', cat_clf),
        ('extra_trees', extra_trees_clf)
    ],
    final_estimator=RandomForestClassifier(random_state=42),
    stack_method='predict_proba'  # Use predict_proba for meta-features
)

# Train the stacking classifier
stacking_clf.fit(X_train_fe, y_train)

# Print classification report for each model
classifiers = {
    'Random Forest': rf_clf,
    'LightGBM': lgbm_clf,
    'CatBoost': cat_clf,
    'Extra Trees': extra_trees_clf,
    'Stacking': stacking_clf
}

for clf_name, clf in classifiers.items():
    y_pred = clf.predict(X_test_fe)
    class_report = classification_report(y_test, y_pred)
    print("Classification Report for", clf_name)
    print(class_report)
    print("----------------------------------------------")


Classification Report for Random Forest
              precision    recall  f1-score   support

           0       0.69      0.81      0.74       160
           1       0.06      0.03      0.04       315
           2       0.55      0.38      0.45       327
           3       0.82      0.68      0.74       100
           4       0.66      0.62      0.64       105
           5       0.70      0.70      0.70       361
           6       0.35      0.28      0.31       610
           7       0.92      0.94      0.93       125
           8       0.63      0.51      0.56       435
           9       0.47      0.53      0.50       595
          10       0.46      0.61      0.52      1186

    accuracy                           0.51      4319
   macro avg       0.57      0.55      0.56      4319
weighted avg       0.49      0.51      0.49      4319

----------------------------------------------
Classification Report for LightGBM
              precision    recall  f1-score   support

          